## Black Box Approach for all 10 genres classification

We aim at training a Neural Network to distinguish between all the different genres

In [1]:
#importing google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#setting the working directory
%cd /gdrive/MyDrive/polimi/NAML/NAML_proj/

/gdrive/MyDrive/polimi/NAML/NAML_proj


In [9]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import random
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import librosa
import os
import copy

tfk = tf.keras
tfkl = tf.keras.layers
seed = 42

The dataset is composed of Mel-Spectrograms of each audio sample, labeled with respect to each genre

In [10]:
dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
        'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
n_genres = 10

for genre, genre_number in genres.items():
    for filename in os.listdir(f'dataset_old/genres/{genre}'):
        songname = f'dataset_old/genres/{genre}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=29.7)
        ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512)
        ps = librosa.power_to_db(ps**2)
        dataset.append( (ps, genre_number) )
    print(str(genre+' done'))

blues done
classical done
country done
disco done
hiphop done
jazz done
metal done
pop done
reggae done
rock done


We split the dataset according to the following composition:


*   70% training set
*   20% validation set
*   10% test set

Maintaining equal proportions amongst classes



In [11]:
order = np.arange(start = 0, stop = 100, step = 1)

np.random.seed(seed)

training_validation = []
test = []

for i in range(n_genres):
  shuffle = np.random.permutation(order)
  for k in range(90):
    training_validation.append(dataset[i*100 + shuffle[k]])
  for m in range(10):
    test.append(dataset[i*100 + shuffle[m+90]])

In [12]:
X_tr_val, Y_tr_val = zip(*training_validation)
X_test, Y_test = zip(*test)

X_tr_val = np.array([x.reshape( (128, 2559, 1) ) for x in X_tr_val])
X_test = np.array([x.reshape( (128, 2559, 1) ) for x in X_test])


Y_tr_val = np.array(tfk.utils.to_categorical(Y_tr_val, n_genres))
Y_test = np.array(tfk.utils.to_categorical(Y_test, n_genres))

The proposed model is composed as a stack of convolutional layer, followed by a Global Average Pooling layer leading to a fully connected section.

Optimization is performed as a Batch version of Adam optimizer algorithm

In [13]:
def build_model(input_shape, n_units):
# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        name = 'conv_1',
        filters=4,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv1_2 = tfkl.Conv2D(
        name = 'conv_1_2',
        filters=8,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        name = 'pool_1',
        pool_size = (2, 2)
    )(conv1_2)

    conv2 = tfkl.Conv2D(
        name = 'conv_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv2_2 = tfkl.Conv2D(
        name = 'conv_2_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv2)
    pool2 = tfkl.MaxPooling2D(
        name = 'pool_2',
        pool_size = (2, 2)
    )(conv2_2)

    conv3 = tfkl.Conv2D(
        name = 'conv_3',
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        name = 'pool_3',
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        name = 'conv_4',
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        name = 'pool_4',
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        name = 'conv_5',
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        name = 'pool_5',
        pool_size = (2, 2)
    )(conv5)

    conv6 = tfkl.Conv2D(
        name = 'conv_6',
        filters=286,
        kernel_size=(1, 1),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    global_average = tfkl.GlobalAveragePooling2D(name = 'GAP')(conv6)
    global_average = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_GAP')(global_average)
    
    classifier_layer = tfkl.Dense(units=64, name='Classifier', activation='relu')(global_average)
    
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier')(classifier_layer)
    classifier_layer_2 = tfkl.Dense(units=32, name='Classifier_2', activation='relu')(classifier_layer)
    classifier_layer_2 = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier_2')(classifier_layer_2)
    output_layer = tfkl.Dense(units=n_units, activation='softmax', name='Output')(classifier_layer_2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model

    return model

In [14]:
metrics_accuracy = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []

n_trials = 10
input_shape = (128, 2559, 1)
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
adaptive_LR = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)

order = np.arange(start = 0, stop = 90, step = 1)
np.random.seed(seed)
#initialization
X_train = copy.deepcopy(X_tr_val[0:700])
Y_train = copy.deepcopy(Y_tr_val[0:700])
X_valid = copy.deepcopy(X_tr_val[0:200])
Y_valid = copy.deepcopy(Y_tr_val[0:200])


for i in range(n_trials):
  model = build_model(input_shape, n_genres)
  
  #random split in training and validation
  
  for j in range(n_genres):
    shuffle = np.random.permutation(order)
    for k in range(70):
      X_train[k] = copy.deepcopy(X_tr_val[i*90 + k])
      Y_train[k] = copy.deepcopy(Y_tr_val[i*90 + k])
    for m in range(20):
      X_valid[m] = copy.deepcopy(X_tr_val[i*90 + m])
      Y_valid[m] = copy.deepcopy(Y_tr_val[i*90 + m])

  standard_history = model.fit(
    x = X_train,
    y = Y_train,
    epochs = 500,
    batch_size = 100,
    validation_data= (X_valid, Y_valid),
    callbacks = [early_stopping, adaptive_LR]
    )
  predictions = model.predict(X_test)
  metrics_accuracy.append(accuracy_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1)))
  metrics_precision.append(precision_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_recall.append(recall_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_f1.append(f1_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  model.save(str('/gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_'+str(i)))
  

Epoch 1/500
7/7 [==============================] - 13s 2s/step - loss: 2.2824 - accuracy: 0.1014 - val_loss: 2.0609 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.1976 - accuracy: 0.1314 - val_loss: 2.0949 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.1629 - accuracy: 0.1557 - val_loss: 2.0444 - val_accuracy: 0.4500 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.1339 - accuracy: 0.1571 - val_loss: 1.8791 - val_accuracy: 0.4500 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 2s/step - loss: 2.0885 - accuracy: 0.2000 - val_loss: 1.9666 - val_accuracy: 0.4300 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 2.0826 - accuracy: 0.2071 - val_loss: 2.0788 - val_accuracy: 0.3650 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 2.0249 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_0/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_0/assets


Epoch 1/500
7/7 [==============================] - 12s 2s/step - loss: 2.2681 - accuracy: 0.1314 - val_loss: 2.2242 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.1373 - accuracy: 0.1600 - val_loss: 2.2172 - val_accuracy: 0.5400 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 2s/step - loss: 2.0951 - accuracy: 0.1643 - val_loss: 2.1366 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 11s 2s/step - loss: 2.0885 - accuracy: 0.1814 - val_loss: 2.0031 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.0103 - accuracy: 0.2200 - val_loss: 1.8434 - val_accuracy: 0.5350 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 1.9894 - accuracy: 0.2300 - val_loss: 1.7800 - val_accuracy: 0.5150 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 2s/step - loss: 1.9553 - accuracy

...

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_8/assets


Epoch 1/500
7/7 [==============================] - 12s 2s/step - loss: 2.3718 - accuracy: 0.1214 - val_loss: 2.3107 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 11s 2s/step - loss: 2.2252 - accuracy: 0.1300 - val_loss: 2.3585 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 2s/step - loss: 2.1995 - accuracy: 0.1429 - val_loss: 2.3427 - val_accuracy: 0.3850 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.1637 - accuracy: 0.1657 - val_loss: 2.2098 - val_accuracy: 0.4200 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.1314 - accuracy: 0.1814 - val_loss: 2.2265 - val_accuracy: 0.4050 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 2s/step - loss: 2.0714 - accuracy: 0.2271 - val_loss: 2.0566 - val_accuracy: 0.4150 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 2s/step - loss: 2.0486 - 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_9/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_9/assets


In [15]:
metrics_accuracy = np.array(metrics_accuracy)
metrics_precision = np.array(metrics_precision)
metrics_recall = np.array(metrics_recall)
metrics_f1 = np.array(metrics_f1)

print(str('mean accuracy = ' + str(metrics_accuracy.mean() )))
print(str('mean precision = ' + str(metrics_precision.mean() )))
print(str('mean recall = ' + str(metrics_recall.mean() )))
print(str('mean f1 = ' + str(metrics_f1.mean() )))

print('***___***')

print(str('accuracy standard deviation = ' + str(metrics_accuracy.std() )))
print(str('precision standard deviation = ' + str(metrics_precision.std() )))
print(str('recall standard deviation = ' + str(metrics_recall.std() )))
print(str('f1 standard deviation = ' + str(metrics_f1.std() )))

print('***___***')

print(str('accuracy range = ' + str(metrics_accuracy.max() - metrics_accuracy.min() )))
print(str('precision range = ' + str(metrics_precision.max() - metrics_precision.min() )))
print(str('recall range = ' + str(metrics_recall.max() - metrics_recall.min() )))
print(str('f1 range = ' + str(metrics_f1.max() - metrics_f1.min() )))

print('***___***')

print(str('min accuracy = ' + str(metrics_accuracy.min() )))
print(str('min precision = ' + str(metrics_precision.min() )))
print(str('min recall = ' + str(metrics_recall.min() )))
print(str('min f1 = ' + str(metrics_f1.min() )))


mean accuracy = 0.541
mean precision = 0.4374337521640153
mean recall = 0.541
mean f1 = 0.46115867589572135
***___***
accuracy standard deviation = 0.15352198539622916
precision standard deviation = 0.1627280480906344
recall standard deviation = 0.15352198539622916
f1 standard deviation = 0.15607626001029298
***___***
accuracy range = 0.5800000000000001
precision range = 0.6706818181818182
recall range = 0.58
f1 range = 0.6069244644244645
***___***
min accuracy = 0.1
min precision = 0.01
min recall = 0.1
min f1 = 0.01818181818181818


In [16]:
print(metrics_accuracy)
print(metrics_precision)
print(metrics_recall)
print(metrics_f1)

[0.61 0.63 0.58 0.57 0.57 0.1  0.5  0.6  0.57 0.68]
[0.51164216 0.468768   0.53915584 0.45405002 0.42271797 0.01
 0.36911825 0.45939394 0.45880952 0.68068182]
[0.61 0.63 0.58 0.57 0.57 0.1  0.5  0.6  0.57 0.68]
[0.5482476  0.53015036 0.49848023 0.47831471 0.47834779 0.01818182
 0.41898194 0.51084673 0.50492929 0.62510628]
